image.png

In [3]:
import tensorflow as tf
import numpy as np # 추가
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report # 추가


In [4]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 로드
train_dir = '/workspace/merge_data/data/img/train'
val_dir = '/workspace/merge_data/data/img/val'

target_size = (224, 224)
batch_size = 32
seed = 42

positive_class = 'happy'  # 'happy' -> 1, 나머지 폴더 -> 0

# helper: 디렉터리를 스캔해서 파일경로와 라벨(1:happy, 0:others) DataFrame 생성
def make_dataframe_from_dir(base_dir, positive_class='happy'):
    rows = []
    for class_name in sorted(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        label = 1 if class_name == positive_class else 0
        for fname in sorted(os.listdir(class_path)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                rows.append({
                    'filepath': os.path.join(class_path, fname),
                    'label': label,
                    'orig_class': class_name
                })
    df = pd.DataFrame(rows)
    return df

# DataFrame 생성
train_df = make_dataframe_from_dir(train_dir, positive_class=positive_class)
val_df = make_dataframe_from_dir(val_dir, positive_class=positive_class)

print("Train samples:", len(train_df), "Val samples:", len(val_df))
print("Train class distribution:\n", train_df['label'].value_counts())
print("Val class distribution:\n", val_df['label'].value_counts())

# ImageDataGenerator (전처리/증강)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_from_dataframe 생성 (class_mode='raw'로 0/1 레이블 반환)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True,
    seed=seed
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False,  # 평가 시 순서 유지
    seed=seed
)


Train samples: 5841 Val samples: 1151
Train class distribution:
 label
0    4348
1    1493
Name: count, dtype: int64
Val class distribution:
 label
0    852
1    299
Name: count, dtype: int64
Found 5841 validated image filenames.
Found 1151 validated image filenames.


In [5]:

# 모델 정의 (원본 코드와 동일)
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

# 학습
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
)



I0000 00:00:1763904507.807330  548972 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 40137 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

...

 34/183 ━━━━━━━━━━━━━━━━━━━━ 5:03 2s/step - accuracy: 0.7157 - auc: 0.5382 - loss: 0.6133

2025-11-23 13:30:21.528696: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:30:21.656572: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:30:21.782704: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:30:22.246982: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:30:22.386223: E external/local_xla/xla/stream_

183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7343 - auc: 0.5791 - loss: 0.5775

2025-11-23 13:37:05.305129: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:37:05.433854: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:37:05.577308: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:37:05.718573: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 13:37:06.122488: E external/local_xla/xla/stream_

183/183 ━━━━━━━━━━━━━━━━━━━━ 514s 3s/step - accuracy: 0.7430 - auc: 0.6197 - loss: 0.5542 - val_accuracy: 0.7428 - val_auc: 0.6485 - val_loss: 0.5629
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 405s 2s/step - accuracy: 0.7571 - auc: 0.7131 - loss: 0.5120 - val_accuracy: 0.7402 - val_auc: 0.6888 - val_loss: 0.5386
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 392s 2s/step - accuracy: 0.7632 - auc: 0.7470 - loss: 0.4912 - val_accuracy: 0.7341 - val_auc: 0.6903 - val_loss: 0.5406
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 396s 2s/step - accuracy: 0.7757 - auc: 0.7764 - loss: 0.4720 - val_accuracy: 0.7116 - val_auc: 0.6823 - val_loss: 0.5702
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.7939 - auc: 0.7905 - loss: 0.4582 - val_accuracy: 0.7289 - val_auc: 0.6768 - val_loss: 0.5571
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.7944 - auc: 0.8138 - loss: 0.4388 - val_accuracy: 0.7446 - val_auc: 0.6891 - val_loss: 0.6843
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 423s 

In [6]:
# --- 검증 데이터셋에 대한 성능 평가 지표 추가 ---
print("--- 검증 데이터셋 성능 평가 시작 ---")

# 1. 검증 데이터셋의 실제 레이블 (True Labels) 추출
if hasattr(validation_generator, 'labels'):
    validation_labels = validation_generator.labels
elif hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
else:
    validation_labels = val_df['label'].values

# 2. 모델 예측 (Predictions) 수행
validation_generator.reset()
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)

# 3. 예측 확률을 이진 클래스 (0 또는 1)로 변환
pred_classes = (pred_probabilities > 0.5).astype(int).flatten()

# 4. classification_report 출력
class_names = ['others', 'happy']
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)
print("--- 검증 데이터셋 성능 평가 완료 ---")



--- 검증 데이터셋 성능 평가 시작 ---


36/36 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step

--- Classification Report (Validation Set) ---

              precision    recall  f1-score   support

      others     0.7741    0.8967    0.8309       852
       happy     0.4634    0.2542    0.3283       299

    accuracy                         0.7298      1151
   macro avg     0.6187    0.5754    0.5796      1151
weighted avg     0.6934    0.7298    0.7003      1151

--- 검증 데이터셋 성능 평가 완료 ---
